In [1]:
from pathlib import Path

import torch
import yaml
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
from stein_mpc.models import ParticleModel

rc('font', **{'family':'serif', 'serif':['Times New Roman']})
rc('font', size=10)
rc('legend', fontsize=10)
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{amsmath}')

/Users/lubaroli/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def plot(
    system,
    states=None,
    rollouts=None,
    path="",
    ax=None,
    cmap="Oranges",
    plot_goal=True,
    **plot_kwargs,
):
    obst_map = system.obst_map.map.T.cpu()
    init_state = system.init_state[:2].cpu()
    target_state = system.target[:2].cpu()
    states = states[..., :2].cpu()

    ax = plt.gca() if ax is None else ax
    ax.set_xlim(0, system.cmap_size[0])
    ax.set_ylim(0, system.cmap_size[1])
    if system.with_obstacle:
        ax.imshow(obst_map, cmap=cmap)
    # plot state
    if states is not None:
        ax.plot(
            system.to_map_coord(states)[..., 0],
            system.to_map_coord(states)[..., 1],
            linewidth=0.5,
        )
    # plot rollouts
    if rollouts is not None:
        # rollouts dimensions are batch x n_policies x timestep x state
        map_rollouts = system.to_map_coord(rollouts[..., :2]).cpu()
        n_pol = map_rollouts.shape[1]
        colour = iter(plt.cm.rainbow(torch.linspace(0, 1, n_pol).numpy()))
        for policy in range(n_pol):
            ax.plot(
                map_rollouts[..., policy, :, 0].T,
                map_rollouts[..., policy, :, 1].T,
                alpha=0.2,
                color=next(colour),
                linewidth=0.5,
            )
    if plot_goal:
        # plot initial state
        ax.scatter(
            system.to_map_coord(init_state)[0],
            system.to_map_coord(init_state)[1],
            marker="o",
            color="r",
            s=15,
        )
        # plot target state
        ax.scatter(
            system.to_map_coord(target_state)[0],
            system.to_map_coord(target_state)[1],
            marker="*",
            color="r",
            s=25,
        )
    return ax

In [18]:
cmap = mpl.cm.bone_r
fig = plt.figure(figsize=(4.2, 2.5))
gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])

config_file = Path("particle_maze_config.yaml")
with config_file.open() as f:
    config_data = yaml.load(f, yaml.FullLoader)

env_params = config_data["env_params"]
system = ParticleModel(**env_params)

# SVMPC plot
timestamp = 51
data_file = Path("/users/lubaroli/Downloads/3389849069/svmpc/data.pkl")
with data_file as f:
    data = torch.load(f, map_location=torch.device('cpu'))
trajectory = data["trajectory"][:timestamp + 1]
rollouts = data["rollouts"][timestamp]
plot(system, trajectory, rollouts, cmap=cmap, ax=ax1)
# inset axes....
axins1 = ax1.inset_axes([0.03, 0.47, 0.7, 0.5])
plot(system, trajectory, rollouts, cmap=cmap, ax=axins1)
# sub region of the original image
x1, x2, y1, y2 = 100, 220, 40, 140
axins1.set_xlim(x1, x2)
axins1.set_ylim(y1, y2)
axins1.set_xticks([], [])
axins1.set_yticks([], [])
rect, lines = ax1.indicate_inset_zoom(axins1, edgecolor="black", linestyle="--")
ax1.text(0.82, 0.05, 'SVMPC', horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes, fontsize=8)
ax1.set_xticks([], [])
ax1.set_yticks([], [])

# SigSVGD plot
timestamp = 33
data_file = Path("/users/lubaroli/Downloads/3389849069/pathsig/data.pkl")
with data_file as f:
    data = torch.load(f, map_location=torch.device('cpu'))
trajectory = data["trajectory"][:timestamp + 1]
rollouts = data["rollouts"][timestamp]
plot(system, trajectory, rollouts, cmap=cmap, ax=ax0)
# inset axes....
axins0 = ax0.inset_axes([0.03, 0.47, 0.7, 0.5])
plot(system, trajectory, rollouts, cmap=cmap, ax=axins0)
# sub region of the original image
x1, x2, y1, y2 = 100, 220, 40, 140
axins0.set_xlim(x1, x2)
axins0.set_ylim(y1, y2)
axins0.set_xticks([], [])
axins0.set_yticks([], [])
rect, lines = ax0.indicate_inset_zoom(axins0, edgecolor="black", linestyle="--")
ax0.text(0.82, 0.05, 'SigSVGD', horizontalalignment='center', verticalalignment='center', transform=ax0.transAxes, fontsize=8)
ax0.set_xticks([], [])
ax0.set_yticks([], [])

# sns.despine()
plt.savefig("trajectories_plot.pdf", bbox_inches="tight")
plt.close()

<ipython-input-18-9ad06724d64b>:29: MatplotlibDeprecationWarning: Passing the minor parameter of set_ticks() positionally is deprecated since Matplotlib 3.2; the parameter will become keyword-only two minor releases later.
  axins1.set_xticks([], [])
<ipython-input-18-9ad06724d64b>:30: MatplotlibDeprecationWarning: Passing the minor parameter of set_ticks() positionally is deprecated since Matplotlib 3.2; the parameter will become keyword-only two minor releases later.
  axins1.set_yticks([], [])
<ipython-input-18-9ad06724d64b>:33: MatplotlibDeprecationWarning: Passing the minor parameter of set_ticks() positionally is deprecated since Matplotlib 3.2; the parameter will become keyword-only two minor releases later.
  ax1.set_xticks([], [])
<ipython-input-18-9ad06724d64b>:34: MatplotlibDeprecationWarning: Passing the minor parameter of set_ticks() positionally is deprecated since Matplotlib 3.2; the parameter will become keyword-only two minor releases later.
  ax1.set_yticks([], [])
<i